## Notes

This notebook tries out some symplectic integration methods as implemented in the pyhamsys package (see https://pypi.org/project/pyhamsys/).

These are quite slow in the context of using the series solution for square duct flow.
If one simply wants a qualitative check, one can expedite computations by using the approximation $w\approx U(1-x^2)(1-y^2)$ (and adjusting the derivatives accordingly).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#from scipy.integrate import solve_ivp

from pyhamsys import HamSys,solve_ivp_sympext

In [ ]:
# Define the square duct Poiseuille flow and its derivatives (using the series solution)
def w(y,N=10,U=10.0):
    M = 1.0
    W = 1.0-y[1]**2
    sign = +1
    for n in range(N):
        kn = (2*n+1)*np.pi/2
        sign *= -1
        temp = 4*sign/(kn**3*np.cosh(kn))
        W += temp*np.cosh(kn*y[0])*np.cos(kn*y[1])
        M += temp
    return W*U/M
def dwdx(y,N=10,U=10.0):
    M = 1.0
    W = 0*y[0] # ensure correct shape...
    sign = +1
    for n in range(N):
        kn = (2*n+1)*np.pi/2
        sign *= -1
        temp = 4*sign/(kn**3*np.cosh(kn))
        W += kn*temp*np.sinh(kn*y[0])*np.cos(kn*y[1])
        M += temp
    return W*U/M
def dwdy(y,N=10,U=10.0):
    M = 1.0
    W = -2*y[1]
    sign = +1
    for n in range(N):
        kn = (2*n+1)*np.pi/2
        sign *= -1
        temp = 4*sign/(kn**3*np.cosh(kn))
        W += -kn*temp*np.cosh(kn*y[0])*np.sin(kn*y[1])
        M += temp
    return W*U/M

In [ ]:
# Begin by solving the spherical particle system
# (This example doesn't require the extended phase space, but is a straightforward check.)

# Specify parameters
U = 10.0
N = 10
dt = 0.5**8
nt = 2**(10+8)
y0 = [0.2,0.8]
dy0 = [0.0,0.0] # equivalent to ex(0),ey(0)
ez0 = -1.0
assert np.isclose(1-dy0[0]**2-dy0[1]**2,ez0**2)
C0 = -0.5*w(y0,N,U)+ez0

# The spherical particle system
def rhs(t,y):
    ez = C0+0.5*w(y,N,U)
    return np.array([y[2],
                     y[3],
                     -0.5*ez*dwdx(y,N,U),
                     -0.5*ez*dwdy(y,N,U)])

# Specify y_dot directly (i.e. without it deriving via the Hamiltonian)
hs = HamSys(ndof=2,y_dot=rhs)
t0,tf = 0.0,dt*nt
sol = solve_ivp_sympext(hs, (t0,tf), np.array(y0+dy0), step=dt)#, check_energy=True)
#print(f"Error in energy : {sol.err}")

# Plot the trajectory
plt.plot(sol.y[0],sol.y[1])
plt.plot([-AR,AR,AR,-AR,-AR],[-1,-1,1,1,-1],'k-')
plt.gca().set_aspect(1.0)
plt.show()

# Examine the preservation of the Hamiltonian
ts = sol.t
ys  = [sol.y[0],sol.y[1]]
dys = [sol.y[2],sol.y[3]]
Ham = lambda y,dy:0.5*(dy[0]**2+dy[1]**2+(C0+0.5*w(y,N,U))**2)
fig = plt.figure(figsize=(10,4))
ax = fig.add_subplot(121)
ax.plot(ts,Ham(ys,dys)-0.5) # Overall change in H
ax = fig.add_subplot(122)
ax.plot(ts[:100],Ham(ys,dys)[:100]-0.5) # View the initial change in H
plt.show()

In [ ]:
# Try the prolate system

# Specify parameters
U = 10.0
AR = 1.0
N = 10
G = 0.6 # gamma=2
fG = (2*G/(1+G))**0.5
dt = 0.5**8
nt = 2**(10+8)
y0 = [0.2,0.8]
dy0 = [0.0,0.0] # equivalent to ex(0),ey(0)
ez0 = -1.0
assert np.isclose(1-dy0[0]**2-dy0[1]**2,ez0**2)
C0 = np.arctanh(fG*ez0)-0.5*(1+G)*fG*w(y0,N,U)

# The prolate particle system
def rhs(t,y):
    fG = (2*G/(1+G))**0.5
    w_ = w(y,N,U)
    Fw = np.cosh(C0+0.5*(1+G)*fG*w_)
    ez = np.tanh(C0+0.5*(1+G)*fG*w_)/fG
    return np.array([y[2]/Fw,
                     y[3]/Fw,
                     -0.5*(1-G)*dwdx(y,N,U)*ez*Fw,
                     -0.5*(1-G)*dwdy(y,N,U)*ez*Fw])

# Try specifying y_dot directly
hs = HamSys(ndof=2,y_dot=rhs)
t0,tf = 0.0,dt*nt
sol = solve_ivp_sympext(hs, (t0,tf), np.array(y0+dy0), step=dt)

# Plot the trajectory
plt.plot(sol.y[0],sol.y[1])
plt.plot([-AR,AR,AR,-AR,-AR],[-1,-1,1,1,-1],'k-')
plt.gca().set_aspect(1.0)
plt.show()

# Examine the preservation of the Hamiltonian
ts = sol.t
ys  = [sol.y[0],sol.y[1]]
dys = [sol.y[2],sol.y[3]]
Fp = np.cosh(C0+0.5*(1+G)*fG*w(ys,N,U))
Vp = 0.5+0.5/G*(0.5*(1-G)*Fp-0.5*(1+G)/Fp)
Ham = lambda y,dy:0.5*(dy[0]**2+dy[1]**2)/Fp+Vp 
fig = plt.figure(figsize=(10,4))
ax = fig.add_subplot(121)
ax.plot(ts,Ham(ys,dys)-0.5)
ax = fig.add_subplot(122)
ax.plot(ts[:100],Ham(ys,dys)[:100]-0.5)
plt.show()

In [ ]:
# Try the oblate system

# Specify parameters
U = 10.0
AR = 1.0
N = 10
G = -0.6 # gamma=1/2
fG = (-2*G/(1+G))**0.5
dt = 0.5**8
nt = 2**(10+8)
y0 = [0.2,0.8]
dy0 = [0.0,0.0] # equivalent to ex(0),ey(0)
ez0 = -1.0
assert np.isclose(1-dy0[0]**2-dy0[1]**2,ez0**2)
C0 = np.arctan(fG*ez0)-0.5*(1+G)*fG*w(y0,N,U)

# The oblate particle system
def rhs(t,y):
    fG = (-2*G/(1+G))**0.5
    w_ = w(y,N,U)
    Fw = np.cos(C0+0.5*(1+G)*fG*w_)
    ez = np.tan(C0+0.5*(1+G)*fG*w_)/fG
    return np.array([y[2]/Fw,
                     y[3]/Fw,
                     -0.5*(1-G)*dwdx(y,N,U)*ez*Fw,
                     -0.5*(1-G)*dwdy(y,N,U)*ez*Fw])

# Try specifying y_dot directly
hs = HamSys(ndof=2,y_dot=rhs)
t0,tf = 0.0,dt*nt
sol = solve_ivp_sympext(hs, (t0,tf), np.array(y0+dy0), step=dt)

# Plot the trajectory
plt.plot(sol.y[0],sol.y[1])
plt.plot([-AR,AR,AR,-AR,-AR],[-1,-1,1,1,-1],'k-')
plt.gca().set_aspect(1.0)
plt.show()

# Examine the preservation of the Hamiltonian
ts = sol.t
ys  = [sol.y[0],sol.y[1]]
dys = [sol.y[2],sol.y[3]]
Fp = np.cos(C0+0.5*(1+G)*fG*w(ys,N,U))
Vp = 0.5+0.5/G*(0.5*(1-G)*Fp-0.5*(1+G)/Fp)
Ham = lambda y,dy:0.5*(dy[0]**2+dy[1]**2)/Fp+Vp 
fig = plt.figure(figsize=(10,4))
ax = fig.add_subplot(121)
ax.plot(ts,Ham(ys,dys)-0.5)
ax = fig.add_subplot(122)
ax.plot(ts[:100],Ham(ys,dys)[:100]-0.5)
plt.show()